# Creador de archivos `.YAML` y `.job` para BOLTZ-2

## 1. Archivo `.csv` a `.YAML`
El objetivo es crear archivos YAML para meterselos como entrada a BOLTZ y generar modelos de las mutaciones.

In [64]:
import pandas as pd

In [65]:
# Cargar el CSV como DataFrame
df = pd.read_csv('Mutaciones.csv')

In [66]:
df.head(2)

,Complejo,Mutación,Cadena1,Cadena1_seq,Cadena2,Cadena2_seq
0,1A22,A.C181A,'A','PTIPLSRLFDNAMLRAHRLHQLAFDTYQEFEEAYIPKEQKYSFLQ...,'B','SKEPKFTKCRSPERETFSCHWTDEVHHGTKNLGPIQLFYTRRNTQ...
1,1A22,A.D170A,'A','PTIPLSRLFDNAMLRAHRLHQLAFDTYQEFEEAYIPKEQKYSFLQ...,'B','SKEPKFTKCRSPERETFSCHWTDEVHHGTKNLGPIQLFYTRRNTQ...


In [67]:
# Limpieza de la tabla

for i in df:
    df[i] = df[i].str.replace("'","")
    df[i] = df[i].str.replace(",","")
    df[i] = df[i].str.replace("}","")
    df[i] = df[i].str.replace("{","")

In [68]:
df.head(2)

,Complejo,Mutación,Cadena1,Cadena1_seq,Cadena2,Cadena2_seq
0,1A22,A.C181A,A,PTIPLSRLFDNAMLRAHRLHQLAFDTYQEFEEAYIPKEQKYSFLQN...,B,SKEPKFTKCRSPERETFSCHWTDEVHHGTKNLGPIQLFYTRRNTQE...
1,1A22,A.D170A,A,PTIPLSRLFDNAMLRAHRLHQLAFDTYQEFEEAYIPKEQKYSFLQN...,B,SKEPKFTKCRSPERETFSCHWTDEVHHGTKNLGPIQLFYTRRNTQE...


In [69]:
# Reasignación de las columnas

df.iloc[1]

complx = df.iloc[:]['Complejo']
mut = df.iloc[:]['Mutación']
cad1 = df.iloc[:]['Cadena1']
seq1  = df.iloc[:]['Cadena1_seq']
cad2  = df.iloc[:]['Cadena2']
seq2  = df.iloc[:]['Cadena2_seq']

complx[1]

'1A22'

In [70]:
print(df.iloc[493][1])
print(df.iloc[493]['Mutación'])

B.V34A
B.V34A


/tmp/ipykernel_17727/2759136670.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(df.iloc[493][1])


In [71]:
print(len(df))

494


In [72]:
seq1[1][-5:]

'GSCGF'

In [73]:
import os

if not os.path.exists('YAML'):
    os.makedirs('YAML')
    
for i in range(len(df)):
    with open(f'./YAML/{complx[i]}_{mut[i]}.yaml', 'w') as salida:
        salida.write(
f"""version: 1
sequences:
    - protein:
        id: {cad1[i]}
        sequence: {seq1[i]}
    - protein:
        id: {cad2[i]}
        sequence: {seq2[i]}"""
)

## Creación de `job.sh`

In [74]:
import os

In [75]:
if not os.path.exists('Models'):
    os.makedirs('Models')

In [76]:
texto = """#!/bin/bash
#SBATCH -c 32
#SBATCH --mem-per-cpu=3952M
#SBATCH --gres=gpu:a100:1
#SBATCH -t 06:00:00
#SBATCH --job-name=Boltz_prueba2      # Nombre del trabajo (aparecerá en squeue)
#SBATCH --mail-type=BEGIN,END,FAIL  # Recibe correos al inicio, final y si falla
#SBATCH --mail-user=victor.monteagudo@icvv.es # ¡REEMPLAZA ESTO CON TU CORREO!

module --force purge
module load cesga/system boltz/2.1.1

"""


In [77]:
dir_YAML = os.listdir('YAML/')

for archivo_yaml in dir_YAML:
    if archivo_yaml.endswith('.yaml'):
        texto = texto + f'boltz predict "/home/csic/sbf/vmh/Boltz2/Luis/22dataset_mutations_YAML/YAML/{archivo_yaml}" ' 
        texto = texto + '--out_dir /home/csic/sbf/vmh/Boltz2/Luis/22dataset_mutations_YAML/Models --output_format pdb '
        texto = texto + '--use_msa_server --write_full_pae --write_full_pde '
        texto = texto + '--use_potentials --recycling_steps 10 --diffusion_samples 25'
        # texto = texto + '--step_scale 1.638 --num_workers 2 --subsample_msa'
        texto = texto + '\n'


In [78]:
with open('job.sh', 'w') as f:
    f.write(texto)

In [79]:
# prueba_job.sh

with open('prueba_job.sh', 'w') as f:
    f.write(
'''#!/bin/bash
#SBATCH -c 32
#SBATCH --mem-per-cpu=3952M
#SBATCH --gres=gpu:a100:1
#SBATCH -t 06:00:00
#SBATCH --job-name=Boltz_prueba2      # Nombre del trabajo (aparecerá en squeue)
#SBATCH --mail-type=BEGIN,END,FAIL  # Recibe correos al inicio, final y si falla
#SBATCH --mail-user=victor.monteagudo@icvv.es # ¡REEMPLAZA ESTO CON TU CORREO!

module --force purge
module load cesga/system boltz/2.1.1

boltz predict "/home/csic/sbf/vmh/Boltz2/Luis/22dataset_mutations_YAML/YAML/1A22_A.I178A.yaml" --out_dir /home/csic/sbf/vmh/Boltz2/Luis/22dataset_mutations_YAML/Models --output_format pdb --use_msa_server --write_full_pae --write_full_pde --use_potentials --recycling_steps 10 --diffusion_samples 25
'''
)